# Classification

## Combine Data

In [80]:
import pandas as pd
import os

# Directory containing your CSV files
csv_directory = 'Data'

# List to hold each dataframe
dataframes = []

#variables
excercise_name = "squat"
camera_position = "side"

# Iterate through each CSV file in the directory
for filename in os.listdir(csv_directory):
    filename = filename.lower().strip()
    if filename.endswith('.csv') and filename.startswith(f'{excercise_name}_{camera_position}'):
        df = pd.read_csv(os.path.join(csv_directory, filename))
        dataframes.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dataframes, ignore_index=True)


## Preprocess Combined Data:

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming 'target_column' is the name of your target variable
X = combined_df.drop('position', axis=1)
y = combined_df['position']

### Delete Some Features (Hands)

In [82]:
strings_to_search = ['Elbow', 'Wrist', 'Pinky', 'leftIndex', 'rightIndex', 'Thumb' , '_z']
columns_to_drop = X.columns[X.columns.str.contains('|'.join(strings_to_search))]
X = X.drop(columns=columns_to_drop)
X.columns

Index(['nose_x', 'nose_y', 'leftEyeInner_x', 'leftEyeInner_y', 'leftEye_x',
       'leftEye_y', 'leftEyeOuter_x', 'leftEyeOuter_y', 'rightEyeInner_x',
       'rightEyeInner_y', 'rightEye_x', 'rightEye_y', 'rightEyeOuter_x',
       'rightEyeOuter_y', 'leftEar_x', 'leftEar_y', 'rightEar_x', 'rightEar_y',
       'leftMouth_x', 'leftMouth_y', 'rightMouth_x', 'rightMouth_y',
       'leftShoulder_x', 'leftShoulder_y', 'rightShoulder_x',
       'rightShoulder_y', 'leftHip_x', 'leftHip_y', 'rightHip_x', 'rightHip_y',
       'leftKnee_x', 'leftKnee_y', 'rightKnee_x', 'rightKnee_y', 'leftAnkle_x',
       'leftAnkle_y', 'rightAnkle_x', 'rightAnkle_y', 'leftHeel_x',
       'leftHeel_y', 'rightHeel_x', 'rightHeel_y', 'leftFootIndex_x',
       'leftFootIndex_y', 'rightFootIndex_x', 'rightFootIndex_y'],
      dtype='object')

### Split

In [83]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Save Scales

In [84]:
import json

means = scaler.mean_.tolist()
stds = scaler.scale_.tolist()

with open(f'means.json', 'w') as f:
    json.dump(means, f)

with open(f'stds.json', 'w') as f:
    json.dump(stds, f)


## Train Model

In [77]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Train a RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = model.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))

# # Save the model using joblib
# import joblib
# joblib.dump(model, f'{excercise_name}_{camera_position}.joblib')

In [85]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


#Check model accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Save the model
# model.save(f'{excercise_name}_{camera_position}.h5')

Epoch 1/100


/opt/homebrew/lib/python3.10/site-packages/keras/src/engine/data_adapter.py:1798: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


246/246 [==============================] - 1s 2ms/step - loss: 0.1929 - accuracy: 0.9377 - val_loss: 0.1618 - val_accuracy: 0.9399
Epoch 2/100
246/246 [==============================] - 0s 861us/step - loss: 0.1434 - accuracy: 0.9518 - val_loss: 0.1480 - val_accuracy: 0.9429
Epoch 3/100
246/246 [==============================] - 0s 762us/step - loss: 0.1282 - accuracy: 0.9547 - val_loss: 0.1349 - val_accuracy: 0.9480
Epoch 4/100
246/246 [==============================] - 0s 707us/step - loss: 0.1221 - accuracy: 0.9546 - val_loss: 0.1304 - val_accuracy: 0.9485
Epoch 5/100
246/246 [==============================] - 0s 712us/step - loss: 0.1162 - accuracy: 0.9570 - val_loss: 0.1197 - val_accuracy: 0.9506
Epoch 6/100
246/246 [==============================] - 0s 692us/step - loss: 0.1107 - accuracy: 0.9573 - val_loss: 0.1279 - val_accuracy: 0.9501
Epoch 7/100
246/246 [==============================] - 0s 811us/step - loss: 0.1065 - accuracy: 0.9583 - val_loss: 0.1102 - val_accuracy: 0.9557

In [86]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open(f'model_{excercise_name}_{camera_position}.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/tmpw7x_9app/assets


INFO:tensorflow:Assets written to: /var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/tmpw7x_9app/assets
2024-06-10 23:26:02.316145: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-06-10 23:26:02.316161: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-06-10 23:26:02.316345: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/tmpw7x_9app
2024-06-10 23:26:02.316907: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-06-10 23:26:02.316912: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/tmpw7x_9app
2024-06-10 23:26:02.318864: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-06-10 23:26:02.347690: I tensorflow/cc/saved_model/loader.cc:215] Running initialization